# Importando as Bibliotecas

In [3]:
import requests
import pandas as pd
import sqlalchemy as db
from dotenv import load_dotenv, find_dotenv
import os

# Ocultando o Serial da API e senha do BD pelo DOTENV

In [118]:
load_dotenv('keys.env')

True

In [9]:
API_KEY = os.getenv('API_KEY')

In [129]:
usuario = os.getenv('usuario')

In [130]:
senha = os.getenv('senha')

# Criando a Classe de Web API e Retornando o DataFrame

In [138]:
class Imdb():
    
    def __init__(self):
        print('Criação para buscar lista dos filmes mais bem avaliados')
    
    def imdb(self):
        '''Criando a conexão através de API do site da Imdb, criando a tabela e editando'''
        
        response = requests.get(f'https://imdb-api.com/en/API/Top250Movies/{API_KEY}')
        response.json()
        response.json().keys()
        results = response.json()
        df = pd.json_normalize(response.json(),'items')

        return df

# Trazendo o DataFrame para a Variável df

In [139]:
df = Imdb()

Criação para buscar lista dos filmes mais bem avaliados


In [140]:
df = df.imdb()

# Manipulando e Modificando o DataFrame

In [143]:
df.columns = ['ID', 'RANK', 'TITULO', 'TITULO COMPLETO', 'ANO DE LANÇAMENTO', 'IMAGEM', 'DIRETORs', 'NOTA DO PUBLICO', 'NUMERO DE VOTOS']
df = df.drop(columns=['IMAGEM'])
new = df["DIRETORs"].str.split("(", n = 1, expand = True)
df["DIRETOR"]= new[0]
new = df["DIRETORs"].str.split(",", n = 1, expand = True)
df["ELENCO"]= new[1]
df = df.drop(columns=['DIRETORs'])
new = df["ID"].str.split('tt', n = 1, expand = True)
df = df.drop(columns=['ID'])
df['ID'] = new[1]
df['ID'] = pd.to_numeric(df['ID'])
df['ANO DE LANÇAMENTO'] = pd.to_numeric(df['ANO DE LANÇAMENTO'])
df['NOTA DO PUBLICO'] = pd.to_numeric(df['NOTA DO PUBLICO'])
df['RANK'] = pd.to_numeric(df['RANK'])
df = df[['ID', 'RANK', 'TITULO', 'TITULO COMPLETO', 'ANO DE LANÇAMENTO', 'DIRETOR', 'ELENCO', 'NOTA DO PUBLICO', 'NUMERO DE VOTOS']]
df['NUMERO DE VOTOS'] = pd.to_numeric(df['NUMERO DE VOTOS'])
df['MEDIA DE VOTOS']=(df['NUMERO DE VOTOS']/df['NUMERO DE VOTOS'].sum())*100

In [144]:
df

,ID,RANK,TITULO,TITULO COMPLETO,ANO DE LANÇAMENTO,DIRETOR,ELENCO,NOTA DO PUBLICO,NUMERO DE VOTOS,MEDIA DE VOTOS
0,111161,1,The Shawshank Redemption,The Shawshank Redemption (1994),1994,Frank Darabont,"Tim Robbins, Morgan Freeman",9.2,2308369,1.727911
1,68646,2,The Godfather,The Godfather (1972),1972,Francis Ford Coppola,"Marlon Brando, Al Pacino",9.1,1593151,1.192541
2,71562,3,The Godfather: Part II,The Godfather: Part II (1974),1974,Francis Ford Coppola,"Al Pacino, Robert De Niro",9.0,1113012,0.833136
3,468569,4,The Dark Knight,The Dark Knight (2008),2008,Christopher Nolan,"Christian Bale, Heath Ledger",9.0,2271307,1.700169
4,50083,5,12 Angry Men,12 Angry Men (1957),1957,Sidney Lumet,"Henry Fonda, Lee J. Cobb",8.9,678646,0.507995
...,...,...,...,...,...,...,...,...,...,...
245,58946,246,The Battle of Algiers,The Battle of Algiers (1966),1966,Gillo Pontecorvo,"Brahim Hadjadj, Jean Martin",8.0,52094,0.038995
246,88247,247,The Terminator,The Terminator (1984),1984,James Cameron,"Arnold Schwarzenegger, Linda Hamilton",8.0,792690,0.593362
247,2991224,248,Tangerines,Tangerines (2013),2013,Zaza Urushadze,"Lembit Ulfsak, Elmo Nüganen",8.0,38774,0.029024
248,2758880,249,Winter Sleep,Winter Sleep (2014),2014,Nuri Bilge Ceylan,"Haluk Bilginer, Melisa Sözen",8.0,44926,0.033629


# Criando a conexão com o BD

In [131]:
class bd():

    def __init__(self):
        print('Criando conexão com o banco de dados')
    
    def conectado(self):
        db_server = 'postgresql'
        user = f'{usuario}'
        password = f'{senha}'
        ip = 'localhost'
        db_name = 'melhores_filmes'
        engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
        conn = engine.connect()
        return conn

# Exportando o DataFrame para o BD

In [132]:
conn = bd()

Criando conexão com o banco de dados


In [133]:
conn = conn.conectado()

In [134]:
df.to_sql(name='lista_de_filmes', con=conn, if_exists='replace', index=False )

# Transformando o DataFrame em .CSV

In [8]:
df.to_csv('/Users/adrianoyoshizawa/Documents/GitHub/projeto_3/data/melhores_filmes.csv', index=False)